In [2]:
import pandas as pd
import pyodbc 

In [3]:
df_cases = pd.read_parquet('./Processed/WHO-COVID19-CASES.parquet')

In [9]:
df_list = df_cases.values.tolist()

In [23]:
server = 'lirkovsrv.database.windows.net'
database = 'lirkovdb'
username = 'lirkov'
password = '{LLirkoff31}'   
driver= '{ODBC Driver 17 for SQL Server}'

In [25]:
conn =  pyodbc.connect('DRIVER='+driver+';SERVER=tcp:'+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = conn.cursor()

for list in df_list:
    cursor.execute("INSERT INTO covid.who_cases (DateReported, Country, IsoCode, WhoRegion, NewCases, CumulativeCases, NewDeaths, CumulativeDeaths) values(?,?,?,?,?,?,?,?)", list)
# for index, row in df.iterrows():
#      cursor.execute("INSERT INTO covid.who_cases (DateReported, Country, IsoCode, WhoRegion, NewCases, CumulativeCases, NewDeaths, CumulativeDeaths) values(?,?,?,?,?,?,?,?)", row.DateReported, row.Country, row.IsoCode, row.WhoRegion, row.NewCases, row.CumulativeCases, row.NewDeaths, row.CumulativeDeaths)
conn.commit()
cursor.close()

In [17]:
for i, y in df_list:
    print(i, y)

ValueError: too many values to unpack (expected 2)